In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_trips = pd.read_excel('PFI_2025_ex2_data.xlsx')

alphas_0 = [1.5, 2, 0.5, -0.5, 0]

def calculate_market_shares(alphas, df_trips=df_trips, increase_param=None, pc=None, return_df=False):
    df_trips = df_trips.copy()
    
    # Quick and dirty way to increase a dataframe parameter
    # for Task 3
    if increase_param is not None:
        df_trips[increase_param] = df_trips[increase_param] * 1.1
        
    if pc is not None:
        df_trips['pc'] = pc
    
    
    params = {
        'k_walk': alphas[0], # Walk constant
        'k_bike': alphas[1], # Bike constant
        'k_car': alphas[2], # Car constant
        'k_carp': alphas[3], # Carpool constant
        'beta_wt': -0.12, # Walk time parameter (U/min)
        'beta_bt': -0.12, # Bike time parameter (U/min)
        'beta_cc': -.05, # Car cost parameter (U/DKK)
        'beta_ct': -0.06, # Car time parameter (U/min)
        'beta_cstat': 1, # Car status parameter
        'beta_cpt': -.1, # Car passenger time parameter (U/min)
        'beta_pc': -0.05, # Public transport cost parameter (U/DKK)
        'beta_pt': -0.05, # Public transport time parameter (U/min)
        'beta_ae': -0.03, # Public transport access/egress parameter (U/min)
        'mu': 0.7, # Logsum parameter
        'alpha': 1, # Size parameter
    }
    
    WALKING_SPEED = 6 # km/h
    BIKING_SPEED = 12 # km/h
    districts = [i for i in range(1, 21)]
    

    def utility_walk(d_from, d_to, alpha=params['k_walk']):
        # Find the on row in df_zones that have
        # ResiZone = d_from
        # DestZone = d_to
        row = df_trips[(df_trips['ResiZone'] == d_from) & (df_trips['DestZone'] == d_to)]
        distance = row['Dist'].values[0]
        return alpha + params['beta_wt'] * 60 * distance/WALKING_SPEED

    def utility_bike(d_from, d_to, alpha=params['k_bike'], df_trips=df_trips):
        # Find the on row in df_zones that have
        # ResiZone = d_from
        # DestZone = d_to
        row = df_trips[(df_trips['ResiZone'] == d_from) & (df_trips['DestZone'] == d_to)]
        distance = row['Dist'].values[0]
        return alpha + params['beta_bt'] * 60 * distance/BIKING_SPEED

    def utility_car(d_from, d_to, alpha=params['k_car']):
        # Find the on row in df_zones that have
        # ResiZone = d_from
        # DestZone = d_to
        # V_n(car|district) = k_car + beta_cc * cc(district) + beta_ct * ct(district) + beta_cstat * CarStat[district]
        district = df_trips[(df_trips['ResiZone'] == d_from) & (df_trips['DestZone'] == d_to)]
        cc = district['cc'].values[0]
        ct = district['ct'].values[0]
        car_stat = district['CarStat'].values[0]
        return alpha + params['beta_cc'] * cc + params['beta_ct'] * ct + params['beta_cstat'] * car_stat

    def utility_carpool(d_from, d_to, alpha=params['k_carp']):
        # Find the on row in df_zones that have
        # ResiZone = d_from
        # DestZone = d_to
        # V_n(carpool|district) = k_carp + beta_carp * ct(district)
        district = df_trips[(df_trips['ResiZone'] == d_from) & (df_trips['DestZone'] == d_to)]
        ct = district['ct'].values[0]
        return alpha + params['beta_cpt'] * ct

    def utility_public_transport(d_from, d_to):
        # Find the on row in df_zones that have
        # ResiZone = d_from
        # DestZone = d_to
        # V_n(pub|district) = beta_pc * pc(district) + beta_pt * pt(district) + beta_ae * ae(district)
        district = df_trips[(df_trips['ResiZone'] == d_from) & (df_trips['DestZone'] == d_to)]
        pc = district['pc'].values[0]
        pt = district['pt'].values[0]
        ae = district['ae'].values[0]
        return alphas[4] + params['beta_pc'] * pc + params['beta_pt'] * pt + params['beta_ae'] * ae

    def destination_utility(d_from, d_to):
        # Calculate the utility for a destination given a starting point
        # V_n(destination) = alpha * ln(Emp(destination) + 0.15 * Pop(destination))
        # Emp(destination) = Employment in destination, EmpDest
        # Pop(destination) = Population in destination, PopDest
        route = df_trips[(df_trips['ResiZone'] == d_from) & (df_trips['DestZone'] == d_to)]
        emp_dest = route['EmpDest'].values[0]
        pop_dest = route['PopDest'].values[0]
        return params['alpha'] * np.log(emp_dest + 0.15 * pop_dest)
    
    
    walking_matrix = pd.DataFrame(index=districts, columns=districts)

    for d_from in districts:
        for d_to in districts:
            walking_matrix.at[d_from, d_to] = utility_walk(d_from, d_to)
        
    # Convert to plottable format
    walking_matrix = walking_matrix.astype(float)
    
    

    biking_matrix = pd.DataFrame(index=districts, columns=districts)

    for d_from in districts:
        for d_to in districts:
            biking_matrix.at[d_from, d_to] = utility_bike(d_from, d_to)
            
    # Convert to plottable format
    biking_matrix = biking_matrix.astype(float)


    car_matrix = pd.DataFrame(index=districts, columns=districts)

    for d_from in districts:
        for d_to in districts:
            car_matrix.at[d_from, d_to] = utility_car(d_from, d_to)
            
    # Convert to plottable format
    car_matrix = car_matrix.astype(float)


    carpool_matrix = pd.DataFrame(index=districts, columns=districts)

    for d_from in districts:
        for d_to in districts:
            carpool_matrix.at[d_from, d_to] = utility_carpool(d_from, d_to)
            
    # Convert to plottable format
    carpool_matrix = carpool_matrix.astype(float)

    # Repeat for public transport
    public_transport_matrix = pd.DataFrame(index=districts, columns=districts)

    for d_from in districts:
        for d_to in districts:
            public_transport_matrix.at[d_from, d_to] = utility_public_transport(d_from, d_to)
            
            
    # Convert to plottable format
    public_transport_matrix = public_transport_matrix.astype(float)


    destination_utilities = pd.DataFrame(index=districts, columns=['Utility'])

    for d_to in districts:
        # Just calculate from district 1, as it is arbitrary    
        destination_utilities.at[d_to, 'Utility'] = destination_utility(1, d_to)

    destination_utilities = destination_utilities.astype(float)

    destination_utilities = destination_utilities.sort_values(by='Utility', ascending=False)


    df_trips['U_walk'] = 0
    df_trips['U_bike'] = 0
    df_trips['U_car'] = 0
    df_trips['U_carpool'] = 0
    df_trips['U_public_transport'] = 0

    for index, row in df_trips.iterrows():
        d_from = row['ResiZone']
        d_to = row['DestZone']
        df_trips.at[index, 'U_walk'] = utility_walk(d_from, d_to)
        df_trips.at[index, 'U_bike'] = utility_bike(d_from, d_to)
        df_trips.at[index, 'U_car'] = utility_car(d_from, d_to)
        df_trips.at[index, 'U_carpool'] = utility_carpool(d_from, d_to)
        df_trips.at[index, 'U_public_transport'] = utility_public_transport(d_from, d_to)

    # Calculate the sum of the exponentials for each district combination
    df_trips['sum_exp'] = df_trips[['U_walk', 'U_bike', 'U_car', 'U_carpool', 'U_public_transport']].apply(lambda x: np.exp(x)).sum(axis=1)

    # Calculate the conditional mode choice probabilities
    # using formula (7.3)
    df_trips['P_walk'] = np.exp(df_trips['U_walk']) / df_trips['sum_exp']
    df_trips['P_bike'] = np.exp(df_trips['U_bike']) / df_trips['sum_exp']
    df_trips['P_car'] = np.exp(df_trips['U_car']) / df_trips['sum_exp']
    df_trips['P_carpool'] = np.exp(df_trips['U_carpool']) / df_trips['sum_exp']
    df_trips['P_public_transport'] = np.exp(df_trips['U_public_transport']) / df_trips['sum_exp']


    # Show to, from and mode choice probabilities
    df_trips[['ResiZone', 'DestZone', 'P_walk', 'P_bike', 'P_car', 'P_carpool', 'P_public_transport']]


    df_trips[(df_trips['ResiZone'] == 1) & (df_trips['DestZone'] == 2)][['P_walk', 'P_bike', 'P_car', 'P_carpool', 'P_public_transport']]


    df_trips["I"] = 0

    for n in districts:
        for d in districts:
            sum_exp = 0
            for m in ["walk", "bike", "car", "carpool", "public_transport"]:
                if m == "walk":
                    sum_exp += np.exp(
                        utility_walk(n, d) / params["mu"]
                    )
                elif m == "bike":
                    sum_exp += np.exp(
                        utility_bike(n, d) / params["mu"]
                    )
                elif m == "car":
                    sum_exp += np.exp(
                        utility_car(n, d) / params["mu"]
                    )
                elif m == "carpool":
                    sum_exp += np.exp(
                        utility_carpool(n, d) / params["mu"]
                    )
                elif m == "public_transport":
                    sum_exp += np.exp(
                        utility_public_transport(n, d) / params["mu"]
                    )
                    
                else:
                    raise ValueError("Unknown mode")
            sum_exp = np.log(sum_exp) * params["mu"]
            
            df_trips.loc[(df_trips["ResiZone"] == n) & (df_trips["DestZone"] == d), "I"] = sum_exp


    for n in districts:
        for d in districts:
            W = destination_utility(n, d)
            
            sum_exp = 0
            for d_prime in districts:
                W_prime = destination_utility(n, d_prime)
                I_prime = df_trips[(df_trips["ResiZone"] == n) & (df_trips["DestZone"] == d_prime)]["I"].values[0]
                sum_exp += np.exp(W_prime + I_prime)
            
            I_nd = df_trips[(df_trips["ResiZone"] == n) & (df_trips["DestZone"] == d)]["I"].values[0]
            
            df_trips.loc[(df_trips["ResiZone"] == n) & (df_trips["DestZone"] == d), "P_dest"] = np.exp(W + I_nd) / sum_exp
            

    P_dest_matrix = pd.DataFrame(index=districts, columns=districts)

    for d_from in districts:
        for d_to in districts:
            P_dest_matrix.at[d_from, d_to] = df_trips[(df_trips['ResiZone'] == d_from) & (df_trips['DestZone'] == d_to)]['P_dest'].values[0]

    # Convert to plottable format
    P_dest_matrix = P_dest_matrix.astype(float)


    travelers = pd.DataFrame(index=districts, columns=districts)

    for d_from in districts:
        for d_to in districts:
            # Get the population in the origin zone
            pop = df_trips[df_trips['ResiZone'] == d_from]['PopResi'].values[0]
            
            # As only half of them are going to work, we divide by 2
            travel_count = pop * 0.5
            
            travelers.at[d_from, d_to] = travel_count
            
    travelers = travelers.astype(float)

    # Multiply the travelers with the destination choice probabilities
    # to get the amount of people going from district i to district j
    travelers = travelers * P_dest_matrix


    # Convert to plottable format
    travelers = travelers.astype(float)

    populations = pd.DataFrame(index=districts, columns=['Population'])

    for d in districts:
        populations.at[d, 'Population'] = df_trips[df_trips['ResiZone'] == d]['PopResi'].values[0]
        
    populations = populations.astype(float)

    populations = populations.sort_values(by='Population', ascending=False)

    walking_amount = pd.DataFrame(index=districts, columns=districts)
    biking_amount = pd.DataFrame(index=districts, columns=districts)
    car_amount = pd.DataFrame(index=districts, columns=districts)
    carpool_amount = pd.DataFrame(index=districts, columns=districts)
    public_transport_amount = pd.DataFrame(index=districts, columns=districts)

    for d_from in districts:
        for d_to in districts:
            # Get the amount of people going from district i to district j
            travel_count = travelers.at[d_from, d_to]

            # Get the mode choice probabilities for the route
            P_walk = df_trips[
                (df_trips["ResiZone"] == d_from) & (df_trips["DestZone"] == d_to)
            ]["P_walk"].values[0]
            P_bike = df_trips[
                (df_trips["ResiZone"] == d_from) & (df_trips["DestZone"] == d_to)
            ]["P_bike"].values[0]
            P_car = df_trips[
                (df_trips["ResiZone"] == d_from) & (df_trips["DestZone"] == d_to)
            ]["P_car"].values[0]
            P_carpool = df_trips[
                (df_trips["ResiZone"] == d_from) & (df_trips["DestZone"] == d_to)
            ]["P_carpool"].values[0]
            P_public_transport = df_trips[
                (df_trips["ResiZone"] == d_from) & (df_trips["DestZone"] == d_to)
            ]["P_public_transport"].values[0]

            # Calculate the amount of people going by each mode
            walking_amount.at[d_from, d_to] = travel_count * P_walk
            biking_amount.at[d_from, d_to] = travel_count * P_bike
            car_amount.at[d_from, d_to] = travel_count * P_car
            carpool_amount.at[d_from, d_to] = travel_count * P_carpool
            public_transport_amount.at[d_from, d_to] = travel_count * P_public_transport

    # Sum the amount of people going by each mode
    walking_amount = walking_amount.astype(float)
    biking_amount = biking_amount.astype(float)
    car_amount = car_amount.astype(float)
    carpool_amount = carpool_amount.astype(float)
    public_transport_amount = public_transport_amount.astype(float)

    # Sum the amount on both axis
    walking_amount = walking_amount.sum(axis=0).sum(axis=0)
    biking_amount = biking_amount.sum(axis=0).sum(axis=0)
    car_amount = car_amount.sum(axis=0).sum(axis=0)
    carpool_amount = carpool_amount.sum(axis=0).sum(axis=0)
    public_transport_amount = public_transport_amount.sum(axis=0).sum(axis=0)


    total_amount = sum([walking_amount, biking_amount, car_amount, carpool_amount, public_transport_amount])

    walking_market_share = walking_amount / total_amount
    biking_market_share = biking_amount / total_amount
    car_market_share = car_amount / total_amount
    carpool_market_share = carpool_amount / total_amount
    public_transport_market_share = public_transport_amount / total_amount
    
    if return_df:
        return df_trips

    return np.array([walking_market_share, biking_market_share, car_market_share, carpool_market_share, public_transport_market_share])


/home/mbg/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [80]:
S_0 = calculate_market_shares(alphas_0)
S_0

/tmp/ipykernel_476561/3605520296.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.1529323082663732' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_walk'] = utility_walk(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:175: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.8264661541331866' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_bike'] = utility_bike(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.3748375923493121' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_car'] = utility_car(d_fro

array([0.05204077, 0.17089137, 0.59924943, 0.09937946, 0.07843898])

In [81]:
actual_market_shares = np.array([0.0403, 0.1419, 0.6232, 0.0883, 0.1063])
actual_market_shares

array([0.0403, 0.1419, 0.6232, 0.0883, 0.1063])

In [82]:
alphas_1 = alphas_0 + np.log(actual_market_shares / S_0)
alphas_1

array([ 1.24432411,  1.8140945 ,  0.53918958, -0.61820534,  0.30394428])

In [83]:
S_1 = calculate_market_shares(alphas_1)
S_1

/tmp/ipykernel_476561/3605520296.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8972564166678425' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_walk'] = utility_walk(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:175: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.6405606561579678' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_bike'] = utility_bike(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.414027170798934' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_car'] = utility_car(d_from

array([0.04027803, 0.14220693, 0.6231271 , 0.08823779, 0.10615015])

In [84]:
alphas_2 = alphas_1 + np.log(actual_market_shares / S_1)
alphas_2

array([ 1.24486936,  1.81193381,  0.53930656, -0.61750051,  0.30535498])

In [85]:
S_2 = calculate_market_shares(alphas_2)
S_0, S_1, S_2

/tmp/ipykernel_476561/3605520296.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8978016726295974' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_walk'] = utility_walk(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:175: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.6383999620414729' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_bike'] = utility_bike(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.4141441551431595' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_car'] = utility_car(d_fro

(array([0.05204077, 0.17089137, 0.59924943, 0.09937946, 0.07843898]),
 array([0.04027803, 0.14220693, 0.6231271 , 0.08823779, 0.10615015]),
 array([0.04029758, 0.14189774, 0.62320439, 0.0883003 , 0.10629999]))

In [86]:
alphas_2

array([ 1.24486936,  1.81193381,  0.53930656, -0.61750051,  0.30535498])

## Task 3

In [87]:
cc_increased = calculate_market_shares(alphas_2, increase_param='cc')
cc_increased

/tmp/ipykernel_476561/3605520296.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8978016726295974' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_walk'] = utility_walk(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:175: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.6383999620414729' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_bike'] = utility_bike(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.4129294182220917' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_car'] = utility_car(d_fro

array([0.04135139, 0.14552078, 0.61435267, 0.09021067, 0.10856449])

In [88]:
ct_increased = calculate_market_shares(alphas_2, increase_param='ct')
ct_increased

/tmp/ipykernel_476561/3605520296.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8978016726295974' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_walk'] = utility_walk(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:175: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.6383999620414729' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_bike'] = utility_bike(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.4128426512991583' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_car'] = utility_car(d_fro

array([0.04156566, 0.14631195, 0.61770405, 0.08491277, 0.10950557])

In [89]:
pc_increased = calculate_market_shares(alphas_2, increase_param='pc')
pc_increased

/tmp/ipykernel_476561/3605520296.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8978016726295974' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_walk'] = utility_walk(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:175: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.6383999620414729' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_bike'] = utility_bike(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.4141441551431595' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_car'] = utility_car(d_fro

array([0.04056125, 0.14288445, 0.62868648, 0.08909259, 0.09877524])

In [90]:
pt_increased = calculate_market_shares(alphas_2, increase_param='pt')
pt_increased

/tmp/ipykernel_476561/3605520296.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8978016726295974' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_walk'] = utility_walk(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:175: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.6383999620414729' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_bike'] = utility_bike(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.4141441551431595' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_car'] = utility_car(d_fro

array([0.04040539, 0.1423041 , 0.62618368, 0.08874033, 0.1023665 ])

In [95]:
# Put the results into a dataframe with columns being 
# the different tests and rows being the different modes
df_task_3 = pd.DataFrame({
    'cc increased': cc_increased,
    'ct increased': ct_increased,
    'pc increased': pc_increased,
    'pt increased': pt_increased
}, index=['Walk', 'Bike', 'Car', 'Carpool', 'Public transport'])

print(df_task_3.to_markdown())

|                  |   cc increased |   ct increased |   pc increased |   pt increased |
|:-----------------|---------------:|---------------:|---------------:|---------------:|
| Walk             |      0.0413514 |      0.0415657 |      0.0405613 |      0.0404054 |
| Bike             |      0.145521  |      0.146312  |      0.142884  |      0.142304  |
| Car              |      0.614353  |      0.617704  |      0.628686  |      0.626184  |
| Carpool          |      0.0902107 |      0.0849128 |      0.0890926 |      0.0887403 |
| Public transport |      0.108564  |      0.109506  |      0.0987752 |      0.102367  |


In [96]:
# Make a similar table where the elasticities are calculated
# as the percentage change in market share divided by the percentage

# Calculate the elasticities
elasticities_cc = 0.1 * (cc_increased - actual_market_shares) / actual_market_shares
elasticities_ct = 0.1 * (ct_increased - actual_market_shares) / actual_market_shares
elasticities_pc = 0.1 * (pc_increased - actual_market_shares) / actual_market_shares
elasticities_pt = 0.1 * (pt_increased - actual_market_shares) / actual_market_shares

# Put the results into a dataframe with columns being
# the different tests and rows being the different modes
# Calculating them as percentages by multiplying by 100 and rounding to 2 decimals
df_task_3_elasticities = pd.DataFrame({
    'cc increased': elasticities_cc * 100,
    'ct increased': elasticities_ct * 100,
    'pc increased': elasticities_pc * 100,
    'pt increased': elasticities_pt * 100
}, index=['Walk', 'Bike', 'Car', 'Carpool', 'Public transport']
).round(2)
# Put percentage sign after the numbers
df_task_3_elasticities = df_task_3_elasticities.astype(str) + '%'

print(df_task_3_elasticities.to_markdown())


|                  | cc increased   | ct increased   | pc increased   | pt increased   |
|:-----------------|:---------------|:---------------|:---------------|:---------------|
| Walk             | 0.26%          | 0.31%          | 0.06%          | 0.03%          |
| Bike             | 0.26%          | 0.31%          | 0.07%          | 0.03%          |
| Car              | -0.14%         | -0.09%         | 0.09%          | 0.05%          |
| Carpool          | 0.22%          | -0.38%         | 0.09%          | 0.05%          |
| Public transport | 0.21%          | 0.3%           | -0.71%         | -0.37%         |


## Task 4

In [ ]:
# Doing the simulation with pc set to 10, to compare with pivoting approach
pc_10 = calculate_market_shares(alphas_2, pc=10)
pc_10, actual_market_shares

/tmp/ipykernel_476561/4148736367.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8978016726295974' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_walk'] = utility_walk(d_from, d_to)
/tmp/ipykernel_476561/4148736367.py:175: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.6383999620414729' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_bike'] = utility_bike(d_from, d_to)
/tmp/ipykernel_476561/4148736367.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.4141441551431595' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_car'] = utility_car(d_fro

(array([0.03889048, 0.1368215 , 0.5962546 , 0.08440043, 0.14363299]),
 array([0.0403, 0.1419, 0.6232, 0.0883, 0.1063]))

In [ ]:
df_model_base = calculate_market_shares(alphas_2, return_df=True)
df_model_sc = calculate_market_shares(alphas_2, pc=10, return_df=True)

df_model_base

/tmp/ipykernel_476561/3605520296.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8978016726295974' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_walk'] = utility_walk(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:175: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.6383999620414729' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_bike'] = utility_bike(d_from, d_to)
/tmp/ipykernel_476561/3605520296.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.4141441551431595' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_trips.at[index, 'U_car'] = utility_car(d_fro

,ResiZone,DestZone,PopResi,EmpResi,PopDest,EmpDest,CarStat,Dist,ae,cc,...,U_carpool,U_public_transport,sum_exp,P_walk,P_bike,P_car,P_carpool,P_public_transport,I,P_dest
0,1,1,15446.270280,8990.436751,15446.270280,8990.436751,0.90,0.289223,6.922697,0.242947,...,-0.639192,-0.664721,12.756230,0.192392,0.403483,0.322428,0.041369,0.040327,2.173873,0.312446
1,1,2,15446.270280,8990.436751,8431.287835,5653.883233,0.90,2.612138,7.759704,2.194196,...,-0.813411,-0.789385,5.686128,0.026577,0.224612,0.590976,0.077969,0.079865,1.435311,0.091343
2,1,3,15446.270280,8990.436751,13526.411712,9921.381329,0.90,4.929963,7.397353,4.141169,...,-0.987248,-0.877850,3.862229,0.002424,0.082308,0.711169,0.096473,0.107626,1.121037,0.115226
3,1,4,15446.270280,8990.436751,8663.696994,5979.914469,0.90,7.037586,7.387384,5.911573,...,-1.145319,-0.967877,3.075050,0.000243,0.029190,0.743576,0.103453,0.123539,0.922762,0.057565
4,1,5,15446.270280,8990.436751,14782.811654,12480.475745,0.90,9.314707,7.908371,7.824354,...,-1.316104,-1.081098,2.505940,0.000019,0.009136,0.748460,0.107017,0.135367,0.727297,0.095593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,20,16,16808.942787,17720.048513,12576.911044,7323.163165,0.99,9.550852,7.605019,8.022716,...,-1.333814,-1.082118,2.632751,0.000014,0.007547,0.763649,0.100074,0.128716,0.786903,0.047014
396,20,17,16808.942787,17720.048513,5608.609071,4747.014808,0.99,7.420676,7.840353,6.233368,...,-1.174051,-0.997885,3.169425,0.000149,0.022505,0.763500,0.097529,0.116317,0.967657,0.034180
397,20,18,16808.942787,17720.048513,1403.393770,9993.609788,0.99,4.944903,7.851580,4.153719,...,-0.988368,-0.892117,4.107685,0.002239,0.076699,0.730695,0.090607,0.099761,1.196355,0.078448
398,20,19,16808.942787,17720.048513,12938.436403,16131.022424,0.99,2.798100,8.199186,2.350404,...,-0.827358,-0.810539,5.762869,0.020978,0.198223,0.627781,0.075865,0.077152,1.469171,0.182512


In [ ]:
# Compare the relative changes in the P_walk, P_bike, P_car, P_carpool and P_public_transport
# between the two dataframes

# Calculate the relative changes
df_relative_change = (
    df_model_sc[["P_walk", "P_bike", "P_car", "P_carpool", "P_public_transport"]]
    - df_model_base[["P_walk", "P_bike", "P_car", "P_carpool", "P_public_transport"]]
) / df_model_base[["P_walk", "P_bike", "P_car", "P_carpool", "P_public_transport"]]

df_relative_change

,P_walk,P_bike,P_car,P_carpool,P_public_transport
0,-0.011324,-0.011324,-0.011324,-0.011324,0.269485
1,-0.022181,-0.022181,-0.022181,-0.022181,0.255545
2,-0.029662,-0.029662,-0.029662,-0.029662,0.245939
3,-0.033899,-0.033899,-0.033899,-0.033899,0.240499
4,-0.037024,-0.037024,-0.037024,-0.037024,0.236485
...,...,...,...,...,...
395,-0.035269,-0.035269,-0.035269,-0.035269,0.238739
396,-0.031980,-0.031980,-0.031980,-0.031980,0.242962
397,-0.027554,-0.027554,-0.027554,-0.027554,0.248645
398,-0.021443,-0.021443,-0.021443,-0.021443,0.256492


In [ ]:
# apply the relative change to the actual OD
df_od = pd.read_excel('PFI_2025_ex2_od.xlsx')

df_od['trip_w_sc'] = df_od['trip_w'] * (1 + df_relative_change['P_walk'])
df_od['trip_b_sc'] = df_od['trip_b'] * (1 + df_relative_change['P_bike'])
df_od['trip_c_sc'] = df_od['trip_c'] * (1 + df_relative_change['P_car'])
df_od['trip_cp_sc'] = df_od['trip_cp'] * (1 + df_relative_change['P_carpool'])
df_od['trip_p_sc'] = df_od['trip_p'] * (1 + df_relative_change['P_public_transport'])

df_od

/home/mbg/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,FromID,ToID,trip_w,trip_b,trip_c,trip_cp,trip_p,trip_w_sc,trip_b_sc,trip_c_sc,trip_cp_sc,trip_p_sc
0,1,1,987.721585,2041.582022,1215.402571,203.684045,141.907615,976.536517,2018.462924,1201.639219,201.377505,180.149584
1,1,2,76.442781,556.483145,998.942044,145.924486,135.584200,74.747232,544.140004,976.784891,142.687790,170.232056
2,1,3,8.540904,197.942473,965.779160,178.100343,166.219377,8.287566,192.071150,937.132444,172.817572,207.099197
3,1,4,1.208151,57.897811,803.238484,112.660213,134.129543,1.167196,55.935147,776.009690,108.841171,166.387503
4,1,5,0.110915,31.357639,923.052825,151.156707,146.088133,0.106809,30.196644,888.877434,145.560235,180.635824
...,...,...,...,...,...,...,...,...,...,...,...,...
395,20,16,0.131775,14.531779,617.614480,90.500037,106.698972,0.127127,14.019255,595.831707,87.308172,132.172162
396,20,17,0.997029,49.967352,584.830000,119.511398,101.353441,0.965143,48.369372,566.126851,115.689365,125.978440
397,20,18,7.688951,174.598898,816.228705,135.064878,123.562731,7.477090,169.788006,793.738365,131.343304,154.286039
398,20,19,84.959797,473.586815,1100.833369,172.875274,154.722094,83.137978,463.431547,1077.227860,169.168256,194.407021


In [97]:
# Calculate how many people are doing each mode
summed = df_od.sum()

# Make a pretty dataframe with pre and post scenario
# Changing row names to be more descriptive
df_summed = pd.DataFrame({
    'Pre scenario': summed[['trip_w', 'trip_b', 'trip_c', 'trip_cp', 'trip_p']].values.round(0),
    'Post scenario': summed[['trip_w_sc', 'trip_b_sc', 'trip_c_sc', 'trip_cp_sc', 'trip_p_sc']].values.round(0)
}, index=['Walk', 'Bike', 'Car', 'Carpool', 'Public transport'])

print(df_summed.to_markdown())

|                  |   Pre scenario |   Post scenario |
|:-----------------|---------------:|----------------:|
| Walk             |          13304 |           13109 |
| Bike             |          46879 |           45970 |
| Car              |         205833 |          194322 |
| Carpool          |          29161 |           27773 |
| Public transport |          35097 |           48523 |
